In [2]:
#Installation blk.
!pip install mysql-connector-python
!pip install sshtunnel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 33.2 MB/s eta 0:00:00


In [3]:
#Import blk.
from   io         import StringIO
import pandas     as pd
import numpy      as np

import re
import mysql.connector
from   mysql.connector import Error

from   sshtunnel import SSHTunnelForwarder


In [7]:
#No IP4 blk

bastion_host   = "18.212.240.245"
bastion_user   = "ec2-user"
bastion_passwd = "147852passwd"
rds_endpoint   = "db-tunnel-end.cdqeagyemzld.us-east-1.rds.amazonaws.com"
rds_port       = 1770
local_port     = 0 #could be everything

# Establishing a tunnel; cuz we wanna a path: this Notebook -> EC2 -> RDS
tunnel = SSHTunnelForwarder(
    (bastion_host, 0),  # If you aint sure 'bout this port, use 0. Default is 22.
    ssh_username = bastion_user,
    ssh_password = bastion_passwd,
    remote_bind_address=(rds_endpoint, rds_port),
    local_bind_address=("localhost", local_port),
    allow_agent=False,
    #set_keepalive=10.0
)

tunnel.start()

try:
  conn = mysql.connector.connect(
        host     = "localhost",
        user     = "walkk",
        password = "123456passwd",
        #database   = "dealercustomers",
        port     = tunnel.local_bind_port
    )

  cursor = conn.cursor()
  cursor.execute("SHOW DATABASES")

  #conn.commit()
  for basis in cursor.fetchall():
    print("\nLOADING...\n")
    print("Fetched: ", basis[0] ,"\n")


  conn.close()
  print( "Successfully fetched and closed RDS. \n")
except Error as e:
  print(f"Error: {e}")

print("Successfully connected and closed EC2. \n")
tunnel.close()


LOADING...

Fetched:  information_schema 


LOADING...

Fetched:  mysql 


LOADING...

Fetched:  performance_schema 


LOADING...

Fetched:  sys 

Successfully fetched and closed RDS. 

Successfully connected and closed EC2. 



#The whole implementation is commensurate with this in Bash:

``ssh user-name@xxx.xxx.xxx.xxx -h something.rds.amazonaws.com -L -N -v``